---
# TF-IDF(Term Frequency-Inverse Document Frequency)
- 텍스트 마이닝과 정보 검색에서 중요한 가중치 기법
- 문서 내의 단어의 중요도를 계산하는 데 사용
- 특정 단어가 문서 집합 내에서 얼마나 중요한지를 평가하는 데 도움을 준다
---

### TF(Term Frequency)-IDF(Inverse Document Frequency), 단어 - 역문서 빈도

- 문서의 유사도, 검색 시스템 등의 기본 알고리즘
- TF-IDF 수치가 높을수록 중요도가 높은 단어
- TF: 단어 빈도
- IDF: 역문서 빈도


In [8]:
import pandas as pd
from math import log

# 문서 준비
docs = [
    "마트 딸기 진짜 비싸다",
    "나는 오늘 아침 사과 먹겠다",
    "아침 사과 금 사과",
    "나는 오늘 저녁 딸기 먹겠다"
]

# 단어 사전 준비
vocab = list(set([word for doc in docs for word in doc.split()]))
vocab.sort()
print(vocab)

['금', '나는', '딸기', '마트', '먹겠다', '비싸다', '사과', '아침', '오늘', '저녁', '진짜']


In [11]:
# 문서 전체 수: N
N = (len(docs))

In [10]:
import pandas as pd
# 문서-단어 행렬, Document-Term Matrix (DTM)
# 문서 별로 특정 단어가 몇 번씩 등장하는지 확인
# 1. 문서를 하나씩 읽어온다.
# 2. 해당 문서에 단어 사전에 있는 단어의 빈도를 확인한다.
# 3. dtm 결과 출력
word_counts_list = []
for doc in docs:
    word_counts = pd.Series(doc.split()).value_counts()
    word_counts_list.append(word_counts)

# 단어 빈도 데이터프레임 생성
df = pd.DataFrame(word_counts_list, columns=vocab).fillna(0).astype(int)
df


,금,나는,딸기,마트,먹겠다,비싸다,사과,아침,오늘,저녁,진짜
count,0,0,1,1,0,1,0,0,0,0,1
count,0,1,0,0,1,0,1,1,1,0,0
count,1,0,0,0,0,0,2,1,0,0,0
count,0,1,1,0,1,0,0,0,1,1,0


In [12]:
# dtm 결과
dtm = []
for idx in range(N):
    doc = docs[idx]
    in_dtm = []
    for v_idx in range(len(vocab)):
        voca = vocab[v_idx]
        in_dtm.append(doc.count(voca))
    dtm.append(in_dtm)

dtm = pd.DataFrame(dtm, columns=vocab)
dtm

,금,나는,딸기,마트,먹겠다,비싸다,사과,아침,오늘,저녁,진짜
0,0,0,1,1,0,1,0,0,0,0,1
1,0,1,0,0,1,0,1,1,1,0,0
2,1,0,0,0,0,0,2,1,0,0,0
3,0,1,1,0,1,0,0,0,1,1,0


##### 문서: d, 단어: t, 문서의 총개수: N
- tf(d,t): 특정 문서 d에서 특정 단어 t의 등장 횟수
- df(t) : 특정 단어 t가 등장한 문서의 수, 특정 문서에 100번 나와도 count = 1 (문서의 개수)
- idf(t) : df(t)에 반비례 하는 수

In [33]:
# tf(d,t): DTM과 동일

def tf(t, d):
    return d.count(t)

# idf(t) = log(n/(df(t)+1))
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [34]:
# tf 결과 실행
result = []
for idx in range(N):
    result.append([])
    doc = docs[idx]
    for v_idx in range(len(vocab)):
        voca = vocab[v_idx]
        result[-1].append(tf(voca, doc))

tf_ = pd.DataFrame(result, columns=vocab)
tf_

,금,나는,딸기,마트,먹겠다,비싸다,사과,아침,오늘,저녁,진짜
0,0,0,1,1,0,1,0,0,0,0,1
1,0,1,0,0,1,0,1,1,1,0,0
2,1,0,0,0,0,0,2,1,0,0,0
3,0,1,1,0,1,0,0,0,1,1,0


In [35]:
# idf 결과 실행
result = []
for v_idx in range(len(vocab)):
    voca = vocab[v_idx]
    result.append(idf(voca))
idx = pd.DataFrame(result, index=vocab)
idx

,0
금,0.693147
나는,0.287682
딸기,0.287682
마트,0.693147
먹겠다,0.287682
비싸다,0.693147
사과,0.287682
아침,0.287682
오늘,0.287682
저녁,0.693147


In [38]:
# tf-idf 결과 실행
result = []
for idx in range(N):
    result.append([])
    doc = docs[idx]
    for v_idx in range(len(vocab)):
        voca = vocab[v_idx]
        result[-1].append(tfidf(voca, doc))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,금,나는,딸기,마트,먹겠다,비싸다,사과,아침,오늘,저녁,진짜
0,0.000000,0.000000,0.287682,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147
1,0.000000,0.287682,0.000000,0.000000,0.287682,0.000000,0.287682,0.287682,0.287682,0.000000,0.000000
2,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.575364,0.287682,0.000000,0.000000,0.000000
3,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000,0.000000,0.287682,0.693147,0.000000


In [83]:
# pip install scikit-learn

In [40]:
# sklearn DTM
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()

# 단어 빈도수 (DTM)
print(vector.fit_transform(docs).toarray())
print(vector.vocabulary_)

[[0 1 1 0 1 0 0 0 0 1]
 [1 0 0 1 0 1 1 1 0 0]
 [0 0 0 0 0 2 1 0 0 0]
 [1 1 0 1 0 0 0 1 1 0]]
{'마트': 2, '딸기': 1, '진짜': 9, '비싸다': 4, '나는': 0, '오늘': 7, '아침': 6, '사과': 5, '먹겠다': 3, '저녁': 8}


In [41]:
# sklearn tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
print(tfidf.fit_transform(docs))
# 문서번호/ 단어 인덱스 / tfidf 가중치

  (0, 4)	0.5254727492640658
  (0, 9)	0.5254727492640658
  (0, 1)	0.41428875116588965
  (0, 2)	0.5254727492640658
  (1, 3)	0.4472135954999579
  (1, 5)	0.4472135954999579
  (1, 6)	0.4472135954999579
  (1, 7)	0.4472135954999579
  (1, 0)	0.4472135954999579
  (2, 5)	0.8944271909999159
  (2, 6)	0.4472135954999579
  (3, 8)	0.5355662725381126
  (3, 3)	0.4222466008506261
  (3, 7)	0.4222466008506261
  (3, 0)	0.4222466008506261
  (3, 1)	0.4222466008506261


In [44]:
print(tfidf.fit_transform(docs).toarray())
print(tfidf.vocabulary_)
print(tfidf.get_feature_names_out())

[[0.         0.41428875 0.52547275 0.         0.52547275 0.
  0.         0.         0.         0.52547275]
 [0.4472136  0.         0.         0.4472136  0.         0.4472136
  0.4472136  0.4472136  0.         0.        ]
 [0.         0.         0.         0.         0.         0.89442719
  0.4472136  0.         0.         0.        ]
 [0.4222466  0.4222466  0.         0.4222466  0.         0.
  0.         0.4222466  0.53556627 0.        ]]
{'마트': 2, '딸기': 1, '진짜': 9, '비싸다': 4, '나는': 0, '오늘': 7, '아침': 6, '사과': 5, '먹겠다': 3, '저녁': 8}
['나는' '딸기' '마트' '먹겠다' '비싸다' '사과' '아침' '오늘' '저녁' '진짜']


In [ ]:
# 실습: 핵심 키워드 추출해 보기
# 1. 필요 데이터 읽어오기
# 2. 형태소 분석 (불용어 제거) 후 명사추출
# 3. 추출된 명사를 기준으로 tf-idf 계산
# 4. 계산된 tf-idf 높은 값을 가진 데이터로 정렬
# 5. 높은 tf-idf 값을 가지는 '단어' 순으로 출력하기

In [68]:
from konlpy.corpus import kolaw
from konlpy.tag import Okt
import re
okt = Okt()

kolaw.fileids()

c = kolaw.open('constitution.txt').read()
print(len(c))
print(c[:500])
c = re.sub('[^가-힣\s]', '', c[:1000])
c = c.replace('\n', '')

stop_words = ['은', '는', '이', '가', '을', '를', '의', '더', '잘', '.', ',', '와', '[0-9]', 'ㄴ','와', '에', '·', '이제', '년', '월','일','안']

word_tokens = okt.nouns(c)

# 불용어 제거
remove_tokens = [word for word in word_tokens if word not in stop_words]
print(remove_tokens)

# 형태소 분석 결과를 하나의 문서로 합치기
combined_doc = ','.join(remove_tokens)
combined_doc

18884
대한민국헌법

유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.

       제1장 총강
  제1조 ① 대한민국은 민주공화국이다.
②대한민국의 주권은 국민에게 있고, 
['대한민국', '헌법', '유구', '역사', '전통', '우리', '국민', '운동', '건립', '대한민국', '임시정부', '법', '통과', '불의', '항거', '민주', '이념', '계승', '조국', '민주', '개혁', '평화', '통일', '사명', '입', '각하', '정의', '인도', '동포', '애', '로써', '민족', '단결', '공고', '모든', '사회', '폐습', '불의', '타파', '자율', '조화', '바탕', '자유민주', '질서', '더욱', '정치', '경제사', '회문', '모든', '영역', '각인', '기회', '능력', '최고', '도로', '발휘', '자유', '권리', '책임', '의무', '완수', '국민', '생활', '향상', '기하', '밖', '항구', '세계', '평화', '인류', '공영', '이바지', '함', '우리', '우리', '자손', '안전', '자유', '행복', '확보', '것', '다

'대한민국,헌법,유구,역사,전통,우리,국민,운동,건립,대한민국,임시정부,법,통과,불의,항거,민주,이념,계승,조국,민주,개혁,평화,통일,사명,입,각하,정의,인도,동포,애,로써,민족,단결,공고,모든,사회,폐습,불의,타파,자율,조화,바탕,자유민주,질서,더욱,정치,경제사,회문,모든,영역,각인,기회,능력,최고,도로,발휘,자유,권리,책임,의무,완수,국민,생활,향상,기하,밖,항구,세계,평화,인류,공영,이바지,함,우리,우리,자손,안전,자유,행복,확보,것,다짐,제정,차,개정,헌법,국회,의결,국민투표,개정,장,강,제조,대한민국,민주공화국,대한민국,주권,국민,모든,권력,국민,제조,대한민국,국민,요건,법률,정,국가,법률,정,바,재외국민,보호,의무,제조,대한민국,영토,한반도,그,부속,도서,제조,대한민국,통일,지향,자유민주,질서,입각,평화,통일,정책,수립,추진,제조,대한민국,국제,평화,유지,노력,침략,전쟁,부인,국군,국가,안전보장,국토,방위,의무,수행,함,사명,그,정치,중립성,준수,제조,헌법,체결,공포,조약,일반,승인,국제,법규,국내법,효력,외국인,국제,법,조약,정,바,그,지위,보장,제조,공무원,국민,전체,대한,봉사자,국민,대하,책임,공무원,신분,정치,중립성,법률,정,바,보장,제조,정당,설립,자유,복수정당제'

In [82]:
# TF-IDF 변환기 생성
tfidf = TfidfVectorizer()

# TF-IDF 행렬 생성
tfidf_matrix = tfidf.fit_transform([combined_doc])

# TF-IDF 행렬의 단어 목록 얻기
feature_names = tfidf.get_feature_names_out()

# TF-IDF 행렬의 단어와 해당 TF-IDF 값 출력 (TF-IDF 값이 높은 순으로 정렬)
sorted_tfidf_indices = tfidf_matrix.toarray().argsort()[0][::-1]  # TF-IDF 값을 기준으로 정렬된 인덱스 배열
for idx in sorted_tfidf_indices[:3]:
    word = feature_names[idx]
    tfidf_value = tfidf_matrix[0, idx]
    print(f"{word}: {tfidf_value}")

대한민국: 0.3903600291794133
제조: 0.3903600291794133
국민: 0.34156502553198664


---
강사님ver

---

In [99]:
from konlpy.tag import _okt
okt = Okt()

# 문서 준비
text = kolaw.open('constitution.txt').read()

# 명사 추출
nouns = okt.nouns(text)
processed_text = " ".join(nouns)[:100]
print(processed_text)

# TF-IDF 벡터
vectorizer = TfidfVectorizer()

# 텍스트를 TF-IDF 벡터로 변환
tfidf_matrix = vectorizer.fit_transform([processed_text])
print(tfidf_matrix)

# 단어와 그에 대한 tf-idf 값 얻기
feature_names = vectorizer.get_feature_names_out()
print(feature_names)
tfidf_values =tfidf_matrix.toarray().tolist()[0]
print(tfidf_values)

# sorting & result
keywords = [feature_names[i] for i in sorted(range(len(tfidf_values)), key=lambda k: tfidf_values[k], reverse=True)]
print(keywords)

대한민국 헌법 유구 역사 전통 우리 국민 운동 건립 대한민국 임시정부 법 통과 불의 항거 민주 이념 계승 조국 민주 개혁 평화 통일 사명 입 각하 정의 인도 동포 애 로써 민족 단
  (0, 8)	0.17407765595569785
  (0, 7)	0.17407765595569785
  (0, 6)	0.17407765595569785
  (0, 17)	0.17407765595569785
  (0, 20)	0.17407765595569785
  (0, 0)	0.17407765595569785
  (0, 11)	0.17407765595569785
  (0, 23)	0.17407765595569785
  (0, 24)	0.17407765595569785
  (0, 1)	0.17407765595569785
  (0, 21)	0.17407765595569785
  (0, 3)	0.17407765595569785
  (0, 16)	0.17407765595569785
  (0, 9)	0.3481553119113957
  (0, 25)	0.17407765595569785
  (0, 10)	0.17407765595569785
  (0, 22)	0.17407765595569785
  (0, 18)	0.17407765595569785
  (0, 2)	0.17407765595569785
  (0, 14)	0.17407765595569785
  (0, 4)	0.17407765595569785
  (0, 13)	0.17407765595569785
  (0, 19)	0.17407765595569785
  (0, 12)	0.17407765595569785
  (0, 15)	0.17407765595569785
  (0, 26)	0.17407765595569785
  (0, 5)	0.3481553119113957
['각하' '개혁' '건립' '계승' '국민' '대한민국' '동포' '로써' '민족' '민주' '불의' '사명' '역사' '우리'
 '운동' '유구' '이념' '인도' '임시정부' 